## Chapter Exercises

### Multiple choice

1. Given the following datatype:

```haskell
data Weekday =
    Monday
  | Tuesday
  | Wednesday
  | Thursday
  | Friday
```
we can say:

- a) Weekday is a type with five data constructors  ✅
- b) Weekday is a tree with five branches
- c) Weekday is a product type
- d) Weekday takes five arguments

2. and with the same datatype definition in mind, what is the type of the following function, `f`?

```haskell
f Friday = "Miller Time"
```
- a) `f :: [Char]`
- b) `f :: String -> String`
- c) `f :: Weekday -> String` ✅
- d) `f :: Day -> Beer`

3. Types defined with the data keyword
- a) must have at least one argument
- b) must begin with a capital letter ✅
- c) must be polymorphic
- d) cannot be imported from modules

4. The function `g xs = xs !! (length xs - 1)`
- a) is recursive and may not terminate
- b) delivers the head of `xs`
- c) delivers the final element of `xs` ✅
- d) has the same type as `xs`

### Ciphers
In the Lists chapter, you wrote a Caesar cipher. Now, we want to
expand on that idea by writing a Vigenère cipher. A Vigenère cipher is another substitution cipher, based on a Caesar cipher, but it
uses a series of Caesar ciphers for polyalphabetic substitution. The
substitution for each letter in the plaintext is determined by a fixed
keyword.
So, for example, if you want to encode the message `“meet at dawn”` the first step is to pick a keyword that will determine which
Caesar cipher to use. We’ll use the keyword `“ALLY”` here. You repeat
the keyword for as many characters as there are in your original
message:
```
MEET AT DAWN
ALLY AL LYAL
```
Now the number of rightward shifts to make to encode each
character is set by the character of the keyword that lines up with it.
The `’A’` means a shift of `0`, so the initial M will remain M. But the `’L’`
for our second character sets a rightward shift of `11`, so `’E’` becomes
`’P’`. And so on, so “meet at dawn” encoded with the keyword `“ALLY”`
becomes `“MPPR AE OYWY”`.

In [ ]:
module Vingere where

import Data.Char


encrypt :: [(Char, Char)] -> String
encrypt =
    map (shift . codeToShift)
    where
        padding                = ord 'A'
        codeToShift (x, code)  = (x, ord code - padding)
        shift (char, shift)    = chr ((((ord char - padding ) + shift ) `mod` 26) + padding)


vingere :: String -> String -> String
vingere text key =
    restoreNonAlpha $ encrypt charAndCode
    where
        cycledKey             = cycle (toUpperString key)
        charAndCode           = zip clearedText cycledKey
        clearedText           = removeNonAlpha (toUpperString text)
        toUpperString         = map toUpper
        removeNonAlpha        = filter isAlpha
        restoreNonAlpha text' = foldl selector "" text
            where
                selector :: String -> Char -> String
                selector acc x = 
                    if not (isAlpha x)
                    then acc ++ [x]
                    else acc ++ [text' !! (length acc - getNonAlpha acc)]
                    where
                        getNonAlpha :: String -> Int
                        getNonAlpha str = 
                            sum $ map (\char -> if isAlpha char then 0 else 1) str

### As-patterns

***As-patterns*** in Haskell are a nifty way to be able to pattern match on
part of something and still refer to the entire original value. Some
examples:

```haskell
f :: Show a => (a, b) -> IO (a, b)
f t@(a, _) = do
    print a
    return t
```

Here we pattern-matched on a tuple so we could get at the first
value for printing, but used the `@` symbol to introduce a binding
named `t` in order to refer to the whole tuple rather than just a part.

```haskell
Prelude> f (1, 2)
1
(1,2)
```

We can use as-patterns with pattern matching on arbitrary data
constructors, which includes lists:

```haskell
doubleUp :: [a] -> [a]
doubleUp [] = []
doubleUp xs@(x:_) = x : xs

Prelude> doubleUp []
[]

Prelude> doubleUp [1]
[1,1]

Prelude> doubleUp [1, 2]
[1,1,2]

Prelude> doubleUp [1, 2, 3]
[1,1,2,3]
```

Use as-patterns in implementing the following functions:

1. This should return True if (and only if) all the values in the first list appear in the second list, though they need not be contiguous.
```haskell
isSubseqOf :: (Eq a)
    => [a]
    -> [a]
    -> Bool
```

In [3]:
import Data.List (elemIndex)

isSubseqOf :: (Eq a)
    => [a]
    -> [a]
    -> Bool

isSubseqOf [] _ = True
isSubseqOf xs@(x:xs') ys =
    case elemIndex x ys of
        Nothing -> False
        Just i -> isSubseqOf xs' (drop i ys)

In [4]:
isSubseqOf "blah" "halbwoot"

False

In [5]:
isSubseqOf "blah" "wootblah"

True

2. Split a sentence into words, then tuple each word with the capitalized form of each.
```haskell
capitalizeWords :: String -> [(String, String)]

Prelude> capitalizeWords "hello world"
[("hello", "Hello"), ("world", "World")]
```

In [10]:
import Data.Char ( toUpper, toLower )


capitalize :: String -> String
capitalize (x:xs) = toUpper x : map toLower xs

capitalizeWords :: String -> [(String, String)]
capitalizeWords text =
    zip (words text) (map capitalize (words text))

In [11]:
capitalizeWords "hello world"

[("hello","Hello"),("world","World")]

### Language exercises

1. Write a function that capitalizes a word.

```haskell
capitalizeWord :: String -> String
capitalizeWord = undefined
```

Example output.

```haskell
Prelude> capitalizeWord "Chortle"
"Chortle"
Prelude> capitalizeWord "chortle"
"Chortle"
```

In [12]:
capitalizeWord :: String -> String
capitalizeWord (x:xs) = toUpper x : map toLower xs

2. Write a function that capitalizes sentences in a paragraph. Recognize when a new sentence has begun by checking for periods. Reuse the `capitalizeWord` function.

In [24]:
splitOn :: Char -> String -> [String]
splitOn c text =
    case nextSubstring of
        [] -> []
        _  -> nextSubstring : splitOn c rest
    where
        nextSubstring = getNext text
        rest = drop (length (takeWhile (== c) text ++ nextSubstring)) text
        getNext = takeWhile (/= c) . dropWhile (== c)

In [49]:
import Data.List (intercalate)

capitalizeParagraph :: String -> String
capitalizeParagraph text = 
    intercalate ". " (map capitalizeSentence sentences) ++ "."
    where
        sentences = splitOn '.' text
        capitalizeSentence sentence =
            case words sentence of
                [] -> ""
                first:rest -> unwords $ capitalizeWord first : rest

In [ ]:
let s = "blah. woot ha. im going home."
capitalizeParagraph s

"Blah. Woot ha. Im going home."

### Phone exercise

Remember old-fashioned phone inputs for writing text where
you had to press a button multiple times to get different letters to
come up? You may still have to do this when you try to search for a
movie to watch using your television remote control. You’re going
to write code to translate sequences of button presses into strings
and vice versa.

So! Here is the layout of the phone:
|  |  |  |
| --- | --- | --- |
| 1 | 2 ABC | 3 DEF |
| 4 GHI | 5 JKL | 6 MNO |
| 7 PQRS | 8 TUV | 9 WXYZ |
| * ^ | 0 + _ | # . , |

Where star `(*)` gives you capitalization of the letter you’re writing
to your friends, and 0 is your space bar. To represent the digit itself,
you press that digit once more than the letters it represents. If you
press a button one more than is required to type the digit, it wraps
around to the first letter. For example:

```
2 -> 'A'
22 -> 'B'
222 -> 'C'
2222 -> '2'
22222 -> 'A'
```
So on and so forth. We’re going to kick this around.

1. Create a data structure that captures the phone layout above. The data structure should be able to express enough of how the layout works that you can use it to dictate the behavior of the functions in the following exercises.

```haskell
-- fill in the rest.
data DaPhone = DaPhone
```

In [58]:
type Button = (Char, String)

newtype DaPhone =
    DaPhone [Button]


daPhone = DaPhone [
    ('1', "1"), ('2', "abc2"), ('3', "def3"),
    ('4', "ghi4"), ('5', "jkl5"), ('6', "mno6"),
    ('7', "pqrs7"), ('8', "tuv8"), ('9', "wxyz9"),
    ('*', ""), ('0', " +_0"), ('#', ".,")]

2. Convert the following conversations into the keypresses required to express them. We’re going to suggest types and functions to fill in order to accomplish the goal, but they’re not obligatory. If you want to do it differently, go right ahead.

```haskell
convo :: [String]
convo =
    ["Wanna play 20 questions",
    "Ya",
    "U 1st haha",
    "Lol ok. Have u ever tasted alcohol",
    "Lol ya",
    "Wow ur cool haha. Ur turn",
    "Ok. Do u think I am pretty Lol",
    "Lol ya",
    "Just making sure rofl ur turn"]

-- validButtons = "1234567890*#"
type Digit = Char

-- Valid presses: 1 and up
type Presses = Int

reverseTaps :: DaPhone
    -> Char
    -> [(Digit, Presses)]
reverseTaps = undefined

-- assuming the default phone definition
-- 'a' -> [('2', 1)]
-- 'A' -> [('*', 1), ('2', 1)]
cellPhonesDead :: DaPhone
    -> String
    -> [(Digit, Presses)]
cellPhonesDead = undefined
```

In [59]:
import Data.Char
import Data.List


convo :: [String]
convo =
    ["Wanna play 20 questions",
    "Ya",
    "U 1st haha",
    "Lol ok. Have u ever tasted alcohol",
    "Lol ya",
    "Wow ur cool haha. Ur turn",
    "Ok. Do u think I am pretty Lol",
    "Lol ya",
    "Just making sure rofl ur turn"]

-- validButtons = "1234567890*#"
type Digit = Char

-- Valid presses: 1 and up
type Presses = Int

reverseTaps :: DaPhone
    -> Char
    -> [(Digit, Presses)]
reverseTaps (DaPhone phoneLayout) char = 
    if isUpper char
    then [('*', 1), taps (toLower char)]
    else [taps char]
    where
        taps c =
            (key, presses)
            where
                presses = presses' + 1
                doesButtonContain (_, ls) = c `elem` ls
                Just (key, letters) = find doesButtonContain phoneLayout
                Just presses' = elemIndex c letters
                
-- assuming the default phone definition
-- 'a' -> [('2', 1)]
-- 'A' -> [('*', 1), ('2', 1)]
cellPhonesDead :: DaPhone
    -> String
    -> [(Digit, Presses)]
cellPhonesDead daPhone = concatMap (reverseTaps daPhone)

In [62]:
map (cellPhonesDead daPhone) convo

[[('*',1),('9',1),('2',1),('6',2),('6',2),('2',1),('0',1),('7',1),('5',3),('2',1),('9',3),('0',1),('2',4),('0',4),('0',1),('7',2),('8',2),('3',2),('7',4),('8',1),('4',3),('6',3),('6',2),('7',4)],[('*',1),('9',3),('2',1)],[('*',1),('8',2),('0',1),('1',1),('7',4),('8',1),('0',1),('4',2),('2',1),('4',2),('2',1)],[('*',1),('5',3),('6',3),('5',3),('0',1),('6',3),('5',2),('#',1),('0',1),('*',1),('4',2),('2',1),('8',3),('3',2),('0',1),('8',2),('0',1),('3',2),('8',3),('3',2),('7',3),('0',1),('8',1),('2',1),('7',4),('8',1),('3',2),('3',1),('0',1),('2',1),('5',3),('2',3),('6',3),('4',2),('6',3),('5',3)],[('*',1),('5',3),('6',3),('5',3),('0',1),('9',3),('2',1)],[('*',1),('9',1),('6',3),('9',1),('0',1),('8',2),('7',3),('0',1),('2',3),('6',3),('6',3),('5',3),('0',1),('4',2),('2',1),('4',2),('2',1),('#',1),('0',1),('*',1),('8',2),('7',3),('0',1),('8',1),('8',2),('7',3),('6',2)],[('*',1),('6',3),('5',2),('#',1),('0',1),('*',1),('3',1),('6',3),('0',1),('8',2),('0',1),('8',1),('4',2),('4',3),('6',2),('

3. How many times do digits need to be pressed for each message?
```haskell
fingerTaps :: [(Digit, Presses)] -> Presses
fingerTaps = undefined
```

In [51]:
fingerTaps :: [(Digit, Presses)] -> Presses
fingerTaps = sum . map snd

4. What was the most popular letter for each message? What was its cost? You’ll want to combine `reverseTaps` and `fingerTaps` to figure out what it cost in taps. `reverseTaps` is a list because you need to press a different button in order to get capitals.

```haskell
mostPopularLetter :: String -> Char
mostPopularLetter = undefined
```

In [65]:
import Data.List (nub)


mostPopularLetter :: String -> Char
mostPopularLetter text = 
    snd $ maximum countsAndChars
    where
        countsAndChars = map count uniques
        count c = (length $ filter (==c) text, c)
        uniques = nub text

In [73]:
mostPopularLetter "Hello, World!"

'l'

### Hutton’s Razor

Hutton’s Razor is a very simple expression language that expresses
integer literals and addition of values in that expression language.
The “trick” to it is that it’s recursive and the two expressions you’re
summing together could be literals or themselves further addition
operations. This sort of datatype is stereotypical of expression languages
used to motivate ideas in research papers and functional
pearls. Evaluating or folding a datatype is also in some sense what
you’re doing most of the time while programming anyway.

1. Your first task is to write the “eval” function which reduces an expression to a final sum.

```haskell
data Expr = 
    Lit Integer
  | Add Expr Expr

eval :: Expr -> Integer
eval = error "do it to it"
```

In [74]:
data Expr = 
    Lit Integer
  | Add Expr Expr

eval :: Expr -> Integer
eval (Lit n) = n
eval (Add x y) = eval x + eval y

In [75]:
eval (Add (Lit 1) (Lit 9001))

9002

2. Write a printer for the expressions.

```haskell
printExpr :: Expr -> String
printExpr = undefined
```

Expected output:

```haskell

Prelude> printExpr (Add (Lit 1) (Lit 9001))
"1 + 9001"
Prelude> let a1 = Add (Lit 9001) (Lit 1)
Prelude> let a2 = Add a1 (Lit 20001)
Prelude> let a3 = Add (Lit 1) a2
Prelude> printExpr a3
"1 + 9001 + 1 + 20001"
```

In [76]:
printExpr :: Expr -> String
printExpr (Lit n) = show n
printExpr (Add x y) = printExpr x ++ " + " ++ printExpr y

In [ ]:
let a1 = Add (Lit 9001) (Lit 1)
let a2 = Add a1 (Lit 20001)
let a3 = Add (Lit 1) a2
printExpr a3

"1 + 9001 + 1 + 20001"